In [1]:
%matplotlib inline

In [2]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from model import *
from variable_optim import *
from noisy_cifar import CIFAR10, CIFAR100

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

==> Preparing data..


In [5]:
def optimizers(net, opti_name, lr, weight_decay, vb=0, num_iters=0):
    if opti_name == 'SGD':
        return optim.SGD(net.parameters(), lr=lr, momentum=0, weight_decay=weight_decay)
    elif opti_name == 'Momentum':
        return optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay, nesterov=False)
    elif opti_name == 'Adam':
        return optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay)
    elif opti_name == 'VSGD-Gaussian':
        return VSGD(net.parameters(), lr=lr, variability=vb, num_iters=num_iters, weight_decay=weight_decay, noise_type='Gaussian')
    elif opti_name == 'VAdam-Gaussian':
        return VAdam(net.parameters(), lr=lr, variability=vb, num_iters=num_iters, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay, noise_type='Gaussian')
    elif opti_name == 'VSGD-Uniform':
        return VSGD(net.parameters(), lr=lr, variability=vb, num_iters=num_iters, weight_decay=weight_decay, noise_type='Uniform')
    elif opti_name == 'VAdam-Uniform':
        return VAdam(net.parameters(), lr=lr, variability=vb, num_iters=num_iters, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay, noise_type='Uniform')
    elif opti_name == 'VSGD-Laplace':
        return VSGD(net.parameters(), lr=lr, variability=vb, num_iters=num_iters, weight_decay=weight_decay, noise_type='Laplace')
    elif opti_name == 'VAdam-Laplce':
        return VAdam(net.parameters(), lr=lr, variability=vb, num_iters=num_iters, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay, noise_type='Laplace')
    else:
        raise('Unspecified optimizer.')
    

In [6]:
criterion = nn.CrossEntropyLoss(reduction='mean')

In [7]:
def train(net, optimizer, epoch):
    print('Epoch: %d' % (epoch+1))
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets, _) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * targets.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print("Training Loss: ", train_loss/total)
    print("Training error:", 1-correct/total)
    return 1 - correct/total, train_loss/total

def test(net):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets, _) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    print("Test error:", 1-correct/total)
    return 1 - correct/total


In [8]:
def define_models(model):
    if model == 'resnet18':
        return ResNet18()
    elif model == 'resnet34':
        return ResNet34()
    elif model == 'vgg16':
        return VGG('VGG16')

In [9]:
trainset = CIFAR10(root='../data/',
                        download=True,
                        train=True,
                        transform=transform_train,
                        noise_type='pairflip',
                        noise_rate=0.4
                       )

testset = CIFAR10(root='../data/',
                       download=True,
                       train=False,
                       transform=transform_test,
                       noise_type='pairflip',
                       noise_rate=0.4
                      )

Files already downloaded and verified
9 10
50000
Actual noise 0.40
[[0.6 0.4 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.6 0.4 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.6 0.4 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.6 0.4 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.6 0.4 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.6 0.4 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.6 0.4 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.6 0.4 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.6 0.4]
 [0.4 0.  0.  0.  0.  0.  0.  0.  0.  0.6]]
Files already downloaded and verified


In [10]:
def inp_peformance(model, learning_rate, batch_size, vb, weight_decay, epochs, N, mode):
    net = define_models(model)
    net = net.to(device)
    if device == 'cuda':
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True
    
    train_err = []
    train_loss = []
    test_err = []
    
    #num_iters = the num of minibatches, if we evaluate our model every epoch
    num_iters = math.ceil(N/ batch_size) 
    optimizer = optimizers(net, mode, learning_rate, weight_decay,vb=vb,num_iters=num_iters)
    
    lambda_lr = lambda epoch: 0.1 ** (epoch // 100)
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_lr)
    
    start_time = time.time()
    print("-"*30+" Mode "+ mode+" starts" )
    for epoch in range(epochs):
        train_err_i, train_loss_i = train(net, optimizer, epoch)
        train_err.append(train_err_i)
        train_loss.append(train_loss_i)
        test_err.append(test(net))
        scheduler.step()
        print ("--- %s seconds ---" % (time.time() - start_time))
        
    save_err({mode:train_loss}, {mode:train_err}, {mode:test_err}, model+'_'+mode, vb, learning_rate, batch_size, weight_decay, epochs, N)
    return train_loss, train_err, test_err

In [11]:
def inp_comparison(model, batch_size, weight_decay, epochs, N):
    train_loss, train_err, test_err = {}, {}, {}
    mode_list =  ['SGD','VSGD-Gaussian','VSGD-Uniform','VSGD-Laplace']
    
    #the learning rates for each optimizers in mode_list
    lr_list = [1., 1., 1., 1.]
    
    #the variability scales for each optimizers in mode_list
    vb_list = [0., 0.05, 0.05, 0.05]
    
    for i,mode in enumerate(mode_list):
        train_loss[mode], train_err[mode], test_err[mode] = inp_peformance(model, lr_list[i], batch_size, vb_list[i], weight_decay, epochs, N, mode)
    return train_loss, train_err, test_err 
    

In [12]:
def plot_figure(model, train_loss, train_err, test_err, batch_size, weight_decay, epochs, N): 
    figure_name = 'LabelNoise40_' + model + '_B'+str(batch_size) + '_N'+ str(N) + '_E' + str(epochs)
    
    plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
    plt.rcParams['image.interpolation'] = 'nearest'
    plt.rcParams['image.cmap'] = 'gray'

    fig = plt.figure()

    axes = plt.gca()
    for key in test_err:
        break
    axes.set_ylim([0., 1.])
    axes.set_xlim([0,epochs])
    colors = ['red','blue','green','orange','brown','yellow','magenta', 'pink','black', 'cyan']
    fix_str1 = 'Test: '
    fix_str2 = 'Train: '
    for idx, mode in enumerate(test_err):
        plt.plot(np.arange(1,epochs+1), test_err[mode], label=fix_str1+mode, ls='solid', linewidth=2, color=colors[idx])
        plt.plot(np.arange(1,epochs+1), train_err[mode], label=fix_str2+mode, ls='dashed', linewidth=2, color=colors[idx])

    plt.ylabel('Errors')
    plt.xlabel('Epochs')
    plt.grid()
    plt.legend()
    plt.show()
    
    fig.savefig('NVRM_'+figure_name + '.png')
    fig.savefig('NVRM_'+figure_name+'.pdf', format='pdf',bbox_inches = 'tight')
    
    
    


In [13]:
def save_err(train_loss, train_err, test_err, model, vb, learning_rate, batch_size, weight_decay, epochs, N):
    csvname = 'LabelNoiseA40_' + model  + '_S'+str(vb) + '_LR'+str(learning_rate) + '_B'+str(batch_size) + '_N'+ str(N) + '_E' + str(epochs)
    current_name = csvname +'.csv'
    
    files_present = glob.glob(current_name)
    if files_present:
        print('WARNING: This file already exists!')
    data_dict = {}
    for mode in test_err:
        data_dict[mode+'_test_err'] = test_err[mode]
        data_dict[mode+'_training_err'] = train_err[mode]
        data_dict[mode+'_training_loss'] = train_loss[mode]
    df = pd.DataFrame(data=data_dict)
    if not files_present:
        df.to_csv(current_name, sep=',', header=True, index=False)
    else:
        print('WARNING: This file already exists!')
        for i in range(1,30):
            files_present = glob.glob(csvname+'_'+str(i)+'.csv')
            if not files_present:
                df.to_csv(csvname+'_'+str(i)+'.csv', sep=',', header=True, index=False)
                return None
    return None
    

In [14]:
batch_size = 128
weight_decay = 1e-4
epochs = 300
model = 'resnet34'

#Adjust the training data size
N = 50000


In [15]:
subset_indices = []
for i in range(N):
    subset_indices.append(int(i*50000/N))
    
trainset_1 = torch.utils.data.Subset(trainset, subset_indices)
trainloader = torch.utils.data.DataLoader(trainset_1, batch_size=batch_size, shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)

In [16]:
train_loss, train_err, test_err = inp_comparison(model, batch_size, weight_decay, epochs, N)

In [17]:
plot_figure(model, train_loss, train_err, test_err, batch_size, weight_decay, epochs, N)